# Abstract

Notebook contains code to import, clean, transform, and export 23 individual data frames containing match data spanning the 00/01 to 22/23 English Premier League seasons.

Data source: https://www.football-data.co.uk/englandm.php

In [1]:
import pandas as pd 
import os
from datetime import datetime 

# Create Individual Path Endings 

In [2]:
start_year = 00  # Change this to the desired start year
end_year = 22  # Change this to the desired end year

# Initialize an empty list to store the generated strings
season_files = []

# Loop to generate the strings
for i in range(start_year, end_year + 1):
    current_year = i
    next_year = (i % 100) + 1
    season_string = f"{current_year:02d}_{next_year:02d}.csv"
    season_string_with_slash = '/' + season_string  # Add a "/" at the beginning
    season_files.append(season_string_with_slash)


# Importing .csv Files

In [5]:
dataframes = {}

base_path = r"/Volumes/Ultra Touch/CofC MATH 540 EPL Project/Datasets" #.csv files stored on external hard drive

for i in season_files:
    path_end = i
    csv_path = base_path + path_end
    
    try:
        dataframe_name = os.path.basename(i).split('.')[0]
        dataframes[dataframe_name] = pd.read_csv(csv_path, encoding = "utf-8", error_bad_lines = False)
    
    except pd.errors.ParserError as e:
            print(f"Error reading CSV file: {i}\nError message: {str(e)}")

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/3268038154.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataframes[dataframe_name] = pd.read_csv(csv_path, encoding = "utf-8", error_bad_lines = False)
/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/3268038154.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataframes[dataframe_name] = pd.read_csv(csv_path, encoding = "utf-8", error_bad_lines = False)
/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/3268038154.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataframes[dataframe_name] = pd.read_csv(csv_path, encoding = "utf-8", error_bad_lines = False)
/var/folders/9c/mcccghjd0j523rgfwsn1ldnm

# Editing Columns 

### Drop Betting Statistic Columns

In [8]:
final_columns_to_keep = ["Date", "HomeTeam", "AwayTeam", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HY", "AY", "HR", "AR"]

In [9]:
for key, df in dataframes.items():
    dataframes[key] = df[final_columns_to_keep]

### Convert "Referee" Values to Last Name Only

In [1]:
dataframes['01_02']
# Formatting in 01_02 data frame is unique, so needs special attention

NameError: name 'dataframes' is not defined

In [12]:
mask = dataframes['01_02']['Referee'].str.contains(',')

# Split and rearrange names for entries in 'LastName, FirstName' format
dataframes['01_02'].loc[mask, 'Referee'] = dataframes['01_02'].loc[mask, 'Referee'].str.split(', ').apply(lambda x: f'{x[1]} {x[0]}')

In [13]:
for key, df in dataframes.items():
    df['Referee'] = df['Referee'].str.split(' ').str[-1]

In [14]:
dataframes['01_02'] = dataframes['01_02'][dataframes['01_02']["Referee"] != '']

In [15]:
for key, df in dataframes.items():
    print(f"DataFrame Name: {key}")
    print("Referees:", df["Referee"].unique())

DataFrame Name: 00_01
Referees: ['Harris' 'Barber' 'Knight' "D'Urso" 'Gallagher' 'Riley' 'Durkin' 'Dunn'
 'Wiley' 'Lodge' 'Poll' 'Halsey' 'Winter' 'Jones' 'Hall' 'Ellaray'
 'Taylor' 'Styles' 'Bennett' 'Barry' 'Dean' 'Burton' 'Messias' 'Wilkes' '']
DataFrame Name: 01_02
Referees: ['Barry' 'Durkin' 'Styles' 'Winter' 'Barber' 'Poll' 'Gallagher' 'Bennett'
 'Rennie' 'Dean' 'Wilkes' 'Knight' "D'Urso" 'Wiley' 'Pugh' 'Riley'
 'Halsey' 'Dunn' 'Wolstenholme' 'Jones' 'Elleray' 'Messias' 'Dowd' 'Foy'
 'Yates']
DataFrame Name: 02_03
Referees: ['Elleray' 'Barber' 'Barry' 'Wiley' 'Poll' 'Bennett' 'Knight' 'Riley'
 "D'Urso" 'Durkin' 'Gallagher' 'Messias' 'Winter' 'Rennie' 'Dean' 'Styles'
 'Wilkes' 'Dunn' 'Pugh' 'Wolstenholme' 'Halsey' 'Foy' 'Dowd']
DataFrame Name: 03_04
Referees: ['Halsey' 'Styles' 'Winter' 'Poll' 'Riley' 'Durkin' 'Barber' 'Dean'
 'Martin' 'Bennett' "D'Urso" 'Barry' 'Messias' 'Rennie' 'Dunn' 'Dowd'
 'Gallagher' 'Wiley' 'Foy' 'Knight' 'Webb' 'Walton']
DataFrame Name: 04_05
Referees: ['

### Standardize "Date" Format

In [17]:
for key, df in dataframes.items():
    # Convert the "Date" column to string to handle different formats
    df['Date'] = df['Date'].astype(str)

    # Define a function to convert the "Date" value to the standard format DD/MM/YY
    def convert_to_standard_format(date_str):
        try:
            date_obj = pd.to_datetime(date_str, format='%d/%m/%Y')
            return date_obj.strftime('%d/%m/%y')
        except ValueError:
            try:
                # Attempt to parse with a different format
                date_obj = pd.to_datetime(date_str, format='%d/%m/%y')
                # Handle cases where the year is ambiguous (adjust this part as needed)
                if date_obj.year <= 99:
                    return date_obj.replace(year=date_obj.year + 2000).strftime('%d/%m/%y')
                return date_obj.strftime('%d/%m/%y')
            except ValueError:
                # Handle other cases, or leave them as is
                return date_str
        
    # Apply the conversion function to the "Date" column
    df['Date'] = df['Date'].apply(convert_to_standard_format)

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/1132233030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = df['Date'].astype(str)
/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/1132233030.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = df['Date'].apply(convert_to_standard_format)


### Create "Day" Column

In [19]:
for key, df in dataframes.items():
    # Convert the "Date" column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')

    # Create the "Day" column containing the day of the week as a string
    df['Day'] = df['Date'].dt.strftime("%A")

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/280372817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/280372817.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Day'] = df['Date'].dt.strftime("%A")


### Create "Month" Column

In [20]:
for key, df in dataframes.items():
    # Ensure the "Date" column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
    
    # Create the "Month" column containing the corresponding month as a full name string
    df['Month'] = df['Date'].dt.strftime("%B")

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/1616367450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/1616367450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Date'].dt.strftime("%B")


### Convert "FTR" (Response) to Binary Values

In [21]:
for key, df in dataframes.items():
    df['FTR'] = df['FTR'].replace({'H': 1, 'D': 0, 'A': 0})

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/1005186426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FTR'] = df['FTR'].replace({'H': 1, 'D': 0, 'A': 0})


# Create COVID Test Set

### "Date" Range = 2020-06-17 to 2021-05-16

In [23]:
COVID_df = pd.DataFrame()

# Define the start and end dates for the desired date range
start_date = '2020-06-17'
end_date = '2021-05-16'

# Assuming dataframes is a dictionary of your 23 DataFrames
dfs_to_concat = []  # Create a list to hold filtered DataFrames

for key, df in dataframes.items():
    # Ensure the "Date" column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter each DataFrame within the date range
    date_range_mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
    filtered_df = df[date_range_mask]

    # Append the filtered DataFrame to the list
    dfs_to_concat.append(filtered_df)

# Concatenate the list of DataFrames into a single combined DataFrame
COVID_df = pd.concat(dfs_to_concat, ignore_index=True)

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/3780378064.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])


In [24]:
COVID_df

,Date,HomeTeam,AwayTeam,FTR,HTHG,HTAG,HTR,Referee,HS,AS,...,HC,AC,HF,AF,HY,AY,HR,AR,Day,Month
0,2020-06-17,Aston Villa,Sheffield United,0.0,0.0,0.0,D,Oliver,14.0,5.0,...,12.0,4.0,11.0,14.0,1.0,1.0,0.0,0.0,Wednesday,June
1,2020-06-17,Man City,Arsenal,1.0,1.0,0.0,H,Taylor,20.0,3.0,...,5.0,2.0,9.0,7.0,1.0,1.0,0.0,1.0,Wednesday,June
2,2020-06-19,Norwich,Southampton,0.0,0.0,0.0,D,Friend,9.0,22.0,...,9.0,7.0,9.0,15.0,1.0,1.0,0.0,0.0,Friday,June
3,2020-06-19,Tottenham,Man United,0.0,1.0,0.0,H,Moss,10.0,12.0,...,7.0,6.0,17.0,18.0,0.0,1.0,0.0,0.0,Friday,June
4,2020-06-20,Watford,Leicester,0.0,0.0,0.0,D,Pawson,8.0,15.0,...,2.0,7.0,15.0,12.0,0.0,1.0,0.0,0.0,Saturday,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2021-05-15,Brighton,West Ham,0.0,0.0,0.0,D,Marriner,10.0,15.0,...,3.0,3.0,7.0,5.0,1.0,1.0,0.0,0.0,Saturday,May
448,2021-05-16,Crystal Palace,Aston Villa,1.0,1.0,2.0,A,Coote,23.0,19.0,...,7.0,7.0,23.0,13.0,3.0,2.0,0.0,0.0,Sunday,May
449,2021-05-16,Tottenham,Wolves,1.0,1.0,0.0,H,Atkinson,24.0,15.0,...,10.0,3.0,8.0,11.0,1.0,0.0,0.0,0.0,Sunday,May
450,2021-05-16,West Brom,Liverpool,0.0,1.0,1.0,D,Dean,10.0,26.0,...,9.0,13.0,7.0,14.0,0.0,0.0,0.0,0.0,Sunday,May


### Remove COVID Test Set Rows from 'dataframes'

In [25]:
start_date = '2020-06-17'
end_date = '2021-05-16'

# Create a dictionary to store the filtered DataFrames
filtered_dataframes = {}

# Assuming dataframes is a dictionary of your 23 DataFrames
for key, df in dataframes.items():
    # Ensure the "Date" column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Create a boolean mask to select rows outside the date range
    date_range_mask = (df['Date'] < start_date) | (df['Date'] > end_date)

    # Filter the DataFrame to retain only the rows outside the date range
    filtered_df = df[date_range_mask]

    # Store the filtered DataFrame in the new dictionary
    filtered_dataframes[key] = filtered_df

/var/folders/9c/mcccghjd0j523rgfwsn1ldnm0000gn/T/ipykernel_59336/1951959242.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])


# Export Data Frames

In [28]:
COVID_df.to_csv("/Volumes/Ultra Touch/CofC MATH 540 EPL Project/Clean Data/COVID.csv", index = False)

In [29]:
export_path = '/Volumes/Ultra Touch/CofC MATH 540 EPL Project/Clean Data/'  

for key, df in filtered_dataframes.items():
    # Construct the full path for the CSV file using the key
    csv_filename = f"{export_path}{key}.csv"

    # Export the DataFrame as a CSV file with the specified name
    df.to_csv(csv_filename, index=False)